<a href="https://colab.research.google.com/github/lejejefr/ECM_2526_FinalProject/blob/main/Scripts/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ce script sert à entrainer notre modèle et à l'utiliser s'il est déjà entrainé


In [2]:
#ce bloc sert à installer les packages python dont nous avons besoin
!pip install pandas numpy scikit-learn matplotlib torch

In [3]:
#dans ce bloc, nous installons les différentes bibliothèques dont nous aurons besoin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split

from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

import torch

In [4]:
# cette fonction calcule l'aire d'une zone rectangulaire en fonction des coordonnées des bords
#lat1, lat2, lon1 et lon2 sont respectivement les latitudes et longitudes des côtés de la zone observée.
#la fonction renvoit une aire en km2

def area(lat1, lat2, lon1, lon2, R = 6378):
    lat1 = np.pi * lat1 / 180.0
    lat2 = np.pi * lat2 / 180.0
    lon1 = np.pi * lon1 / 180.0
    lon2 = np.pi * lon2 / 180.0

    h = max(np.sin(lat1), np.sin(lat2)) - min(np.sin(lat1), np.sin(lat2))
    r = (max(lon1, lon2) - min(lon1,lon2)) / (2 * np.pi)

    return 2 * np.pi * (R ** 2) * h * r

print(area(41.644818,42.022548,-87.904123,-87.525759))
#-->le rectangle couvert par les données est de 1319.5307651175892km2, soit environ 2 fois l'air de chicago. Si on regarde une carte de la ville, on voit que le plus petit
#rectangle contenant la ville peut la contenir à peu près 2 fois, donc a priori la fonction calculant l'aire en fonction des coordonnées fonctionne bien.

1319.5307651175892


In [4]:
#ce bloc permet d'importer la base de donnée propre
url = "https://raw.githubusercontent.com/lejejefr/ECM_2526_FinalProject/refs/heads/main/Data/CleanData.csv"
def get_data_frame_from_url(_url : str = url):
  df = pd.read_csv(_url)
  df = df.dropna()
  df.describe()
  return df
#la ligne suivante sert uniquement à vérifier que la base de données a bien été importée
get_data_frame_from_url()
#-->

,ID,time,day of week,day of year,latitude,longitude
0,28910,62340,0,279,41.750334,-87.665045
1,28909,62100,0,279,41.880968,-87.737940
2,28908,33420,0,279,41.882208,-87.767204
3,28907,12960,0,279,41.821964,-87.690298
4,28906,74100,5,277,41.866235,-87.720517
...,...,...,...,...,...,...
13935,638,58920,4,5,41.748514,-87.664947
13936,636,81000,3,4,41.865156,-87.713732
13937,643,79500,3,4,41.811132,-87.620788
13938,635,54600,0,1,41.924488,-87.699933


Dans cette partie, nous avons mis en place différents modèles KDE pour évaluer la densité de probabilité qui correspond à notre distribution

In [5]:
#kernels possibles : {‘gaussian’, ‘tophat’, ‘epanechnikov’, ‘exponential’, ‘linear’, ‘cosine’}
def get_train_test_data_frames(X, _test_size : float = 0.2):
  return train_test_split(X, test_size=_test_size, random_state=1)

def get_trained_KDE(train_data, _kernel : str = 'gaussian',):
  model = KernelDensity(kernel=_kernel).fit(train_data)
  return model

In [6]:
def train_estimator(estimator, data):
  trained_estimator = estimator.fit(data)
  return trained_estimator

def test_estimator(estimator, data, _test_size : float = 0.2):
  train_data, test_data = train_test_split(data, test_size = _test_size)
  trained_estimator = train_estimator(estimator,train_data)
  score = trained_estimator.score(test_data)
  print(score)

test_estimator(KernelDensity(),get_data_frame_from_url())
#--> score de likelyhood pas fou

-281356906932.5914


In [7]:
def get_best_KDE_generic_algorithm(data, kernel : str ='gaussian', required_columns = [],test_size : float=  0.2): #marche pas x'(
  df = data[required_columns]
  min = []
  max = []
  for column in range(len(required_columns)):
    min.append(df[required_columns[column]].min())
    max.append(df[required_columns[column]].max())
  df = (df-df.min())/(df.max()-df.min())
  X_train, X_test = train_test_split(df,test_size=test_size)

  def KDE_score(bandwidth):
    model = KernelDensity(bandwidth=bandwidth, kernel = kernel)
    model.fit(X_train)
    return model.score(X_test)

  bandwidths = [random.random() for j in range(16)]

  for i in range(10):
    new_bandwidths = []
    for band in bandwidths:
      new_bandwidths.append(band)
      for child in range(2):
        new_bandwidths.append(band + ((2.0 * random.random() - 1.0) / 2.0) * .1)
    new_bandwidths.sort(key=KDE_score)
    bandwidths = new_bandwidths[:16]

  return bandwidths
def get_best_KDE_silvermann(data, kernel : str ='gaussian', required_columns = [],test_size : float=  0.2):
  df = data[required_columns]
  min = []
  max = []
  for column in range(len(required_columns)):
    min.append(df[required_columns[column]].min())
    max.append(df[required_columns[column]].max())
  df = (df-df.min())/(df.max()-df.min())
  X_train, X_test = train_test_split(df,test_size=test_size)

  model = KernelDensity(bandwidth="silverman", kernel = kernel)
  model.fit(X_train)
  return model.score(X_test)

def get_best_KDE_scott(data, kernel : str ='gaussian', required_columns = [],test_size : float=  0.2):
  df = data[required_columns]
  min = []
  max = []
  for column in range(len(required_columns)):
    min.append(df[required_columns[column]].min())
    max.append(df[required_columns[column]].max())
  df = (df-df.min())/(df.max()-df.min())
  X_train, X_test = train_test_split(df,test_size=test_size)

  model = KernelDensity(bandwidth="scott", kernel = kernel)
  model.fit(X_train)
  return model.score(X_test)


#--> silvermann donne en moyenne des résultats meilleurs
for i in range(1):
  print("--------------------------------------")
  print(get_best_KDE_silvermann(get_data_frame_from_url(), required_columns=['time', 'day of week', 'day of year', 'latitude', 'longitude']))
  print(get_best_KDE_scott(get_data_frame_from_url(), required_columns=['time', 'day of week', 'day of year', 'latitude', 'longitude']))

--------------------------------------
-3353.054079352756
-3769.9171331963335


In [8]:
class NormalizedKDE:
  #cette classe sert à normaliser les données en choisissant des facteurs d'échelle personnalisés.
  #on va stocker les minimums et maximums des données d'entrainement et fit un modèle KDE sur les données normalisées
  #on utilise une normalisation linéaire pour chaque paramètre de l'échelle [X.min(), X.max()] vers [0, scale_factor[paramètre]]
  #on palie ainsi à un problème majeur de la class sklearn KernelDensity : l'impossibilité de choisir le paramètre bandwith indépendamment pur chaque paramètre.

  #la fonction d'initialisation permet notamment d'initialiser le model KDE, avec les paramètres principaux : kernel et bandwith
  def __init__(self, kernel="gaussian", bandwidth="silverman", scale_factors = {}):
    self.minimums = None
    self.maximums = None
    self.scale_series = None

    self.scale_factors = scale_factors
    self.model = KernelDensity(
      kernel=kernel,
      bandwidth=bandwidth
      )

  #cette fonction sert à fitter le KDE.
  #au préalable, les minimums, les maximums et la scale_series sont initialisés.
  #ces variables vont servir à normaliser les données de test sur l'échelle des données d'entrainement
  def fit(self, X, y=None):
    self.minimums = X.min()
    self.maximums = X.max()
    self.scale_series = pd.Series(data = self.scale_factors, index = X.columns).fillna(1.0)

    normalized_df = self.scale_series*(X - self.minimums) / (self.maximums - self.minimums)

    self.model.fit(normalized_df)

    return self

  #calcul du score sur les données d'entrainement normalisées
  def score(self, X):
    normalized_df = self.scale_series*(X - self.minimums) / (self.maximums - self.minimums)
    return self.model.score(normalized_df)

  #calcul du score sur les données d'entrainement normalisées par sample
  def score_samples(self, X):
    normalized_df = self.scale_series*(X - self.minimums) / (self.maximums - self.minimums)
    return self.model.score_samples(normalized_df)

  #calcul de la densité sur les données d'entrainement normalisées par sample
  def samples_density(self, X):
    return np.exp(self.score_samples(X))
mod = NormalizedKDE()
x_train,X_test = train_test_split(get_data_frame_from_url()[['time', 'day of week', 'day of year', 'latitude', 'longitude']],test_size = 0.2)
mod.fit(x_train)
print(mod.score(X_test))
print(mod.score_samples(X_test).mean())

-3349.2646433401605
-1.2013144344835582


In [25]:
class BoostedNormalizedKDE(torch.nn.Module):
  def __init__(self, input_dimension = 5, hidden_dimensions = [], base_models = [NormalizedKDE()]):
    super().__init__()
    self.base_models = base_models
    input = input_dimension
    layers = []
    for dimension in hidden_dimensions:
      layers.append(torch.nn.Linear(input, dimension))
      layers.append(torch.nn.ReLU())
      input = dimension
    layers.append(torch.nn.Linear(input, len(base_models)))
    self.net = torch.nn.Sequential(*layers)

  def forward(self, X):
    X_tensor = torch.tensor(X.to_numpy(), dtype=torch.float32)
    return torch.softmax(self.net(X_tensor), dim=1)

  def fit(self, X):
    for model in self.base_models:
      model.fit(X)

  def score_samples(self,X):
    model_sample_score = np.array([model.score_samples(X) for model in self.base_models]).T
    print(np.isna(model_sample_score).count())

    weights = self.forward(X).detach().numpy()
    print(np.isna(weights).count())

    weighted_scores_per_sample = np.sum(model_sample_score * weights, axis=1)
    print(np.isna(weighted_scores_per_sample).count())

    return np.log(weighted_scores_per_sample)
#‘tophat’, ‘epanechnikov’, ‘exponential’, ‘linear’, ‘cosine’
mod = BoostedNormalizedKDE(base_models=[
    NormalizedKDE(kernel="gaussian"),
    NormalizedKDE(kernel="tophat"),
    NormalizedKDE(kernel="epanechnikov"),
    NormalizedKDE(kernel="exponential"),
    NormalizedKDE(kernel="linear"),
    NormalizedKDE(kernel="cosine"),
])
x_train,X_test = train_test_split(get_data_frame_from_url()[['time', 'day of week', 'day of year', 'latitude', 'longitude']],test_size = 0.2)
mod.fit(x_train)
print(mod.score_samples(X_test))

[-1.78555309         nan -2.67504546 ... -1.02848889 -0.96669007
 -0.48573479]


/tmp/ipython-input-3339692385.py:27: RuntimeWarning: invalid value encountered in log
  return np.log(weighted_scores_per_sample)
